In [9]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=6
println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],2))
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# unit sphere constraint
u=2
p=floor(Int64,n/u)+1

I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
vecf=[generate_random_poly(I[j]) for j in 1:p]
f=sum(vecf)

g=[1.0-sum(x[I[j]].^2) for j in 1:p]
J=[j:j for j in 1:p]

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/7)

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{true,Float64}}(undef,l)

randx=2*rand(Float64,n).-1
for j in 1:p
    randx[I[j]]=randx[I[j]]./norm(randx[I[j]])
    randx[I[j]]=rand(Float64,1)[1]*randx[I[j]]
end

for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make constraints feasible
    end
end

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=6
Number of inequality constraints: m=4
Number of equality constraints: l=1


In [10]:
include("../src/ctpPOP.jl")
using .ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=true);

In [11]:
k=2

println("Relaxed order: k=",k)
println("====================")
t=2

println("Term sparse order: t=",t)

Relaxed order: k=2
Term sparse order: t=2


In [12]:
using TSSOS


@time begin
    ~,~,data=cs_tssos_first(Vector{SparseMatrixCSC{UInt8,UInt32}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],n,k,[dg;dh],numeq=l,CS="MD",TS="block",CTP=false);
    for j in 1:t-1
        ~,~,data=cs_tssos_higher!(data,TS="block");
    end
end

------------------------------------------------------
The clique sizes of varibles:
[3, 2]
[2, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 53              
  Cones                  : 0               
  Scalar variables       : 12              
  Matrix variables       : 13              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 

(-1.5027526879732798, nothing, TSSOS.mdata_type(6, 0, 5, [2, 2, 2, 2, 2], SparseMatrixCSC{UInt8,UInt32}[
  [1,  1]  =  0x02
  [1,  2]  =  0x01
  [2,  2]  =  0x01
  [2,  3]  =  0x02
  [2,  4]  =  0x01
  [3,  4]  =  0x01
  [2,  5]  =  0x01
  [4,  5]  =  0x01
  [3,  6]  =  0x02
  [3,  7]  =  0x01
  [4,  7]  =  0x01
  [4,  8]  =  0x02
  [4,  9]  =  0x01
  [5,  9]  =  0x01
  [4, 10]  =  0x01
  [6, 10]  =  0x01
  [5, 11]  =  0x02
  [5, 12]  =  0x01
  [6, 12]  =  0x01
  [6, 13]  =  0x02, 
  [1, 1]  =  0x02
  [2, 2]  =  0x02, 
  [2, 1]  =  0x02
  [3, 2]  =  0x02
  [4, 3]  =  0x02, 
  [4, 1]  =  0x02
  [5, 2]  =  0x02
  [6, 3]  =  0x02, 
  [6, 1]  =  0x02, 
  [6, 1]  =  0x02], Array{Float64,1}[[-0.2044527852390603, -0.6885604086046793, -0.09204237899655343, 0.37861323857884654, 0.37557832898297194, -0.8005519687531542, -0.14242537661318577, 0.9297262261958363, -0.505800311249986, 0.017370710195381278, -0.14696385743411255, 0.24475032163427102, -0.599465392224074], [-1.0, -1.0, 1.0], [-1.0, -1.0

In [13]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val1=ctpPOP.POP_mix_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
    maxit=Int64(1e10),tol=1e-2,use_eqcons_to_get_constant_trace=false,check_tol_each_iter=true)

------------------------------------------------------
The clique sizes of varibles:
[3, 2]
[2, 1]
------------------------------------------------------
  Number of cliques: p=3
  Largest clique size: u=3


  Largest constant trace: a=5.0
  Number of blocks: omega=14
  Number of equality consraints: zeta=67
  Size of the largest block: s^max=7
Modeling time:
  0.877344 seconds (1.13 M allocations: 55.748 MiB)
iter=1   val=-5.725731422078165   gap=2.64113555526334e-16   feas=10.256090236914911
iter=2   val=-1.9085771406927217   gap=0.7144368750511796   feas=3.035584217105014
iter=4   val=-2.546685588530511   gap=0.23951607934645383   feas=3.427988793272919
iter=8   val=-1.5091949584102378   gap=0.45071956161755056   feas=0.8462720374358751
iter=16   val=-1.615104155093557   gap=0.271270269884197   feas=0.6881577494017458
iter=32   val=-1.4904698720043656   gap=0.2506846369865074   feas=0.34912793389601493
iter=64   val=-1.490332628818983   gap=0.13508453626863845   feas=0.12799344780228664
iter=128   val=-1.4995161030424655   gap=0.14442278453514457   feas=0.06652626582791124
iter=256   val=-1.5047124276460786   gap=0.09207028352264028   feas=0.037450798036724936
iter=512   val=-1.50042676

-1.5024557939071583

In [14]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val=ctpPOP.POP_mix_LMBM(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
    tol=1e-2,use_eqcons_to_get_constant_trace=false)

------------------------------------------------------
The clique sizes of varibles:
[3, 2]
[2, 1]
------------------------------------------------------
  Number of cliques: p=3
  Largest clique size: u=3


  Largest constant trace: a=5.0
  Number of blocks: omega=14
  Number of equality consraints: zeta=67
  Size of the largest block: s^max=7
Modeling time:
  0.825282 seconds (1.13 M allocations: 55.739 MiB, 6.35% gc time)
**LMBM solver:
---------------
| Parameters: |
---------------
n:       67
maxtime: 300000.000000
na:      2
mcu:     5
mc:      7
rpar: 
ipar: 
 Entry to LMBM:
 NIT=    1  NFE=    1  F= 0.93258688D+00  WK= 0.1055D-01  QK= 0.5275D-02
 NIT=    2  NFE=    2  F= 0.92203696D+00  WK= 0.1055D-01  QK= 0.5275D-02
 NIT=    3  NFE=    3  F= 0.91148703D+00  WK= 0.1055D-01  QK= 0.5275D-02
 NIT=    4  NFE=    4  F= 0.90093710D+00  WK= 0.1055D-01  QK= 0.5275D-02
 NIT=    5  NFE=    5  F= 0.89038718D+00  WK= 0.1055D-01  QK= 0.5275D-02
 NIT=    6  NFE=    6  F= 0.87983725D+00  WK= 0.1055D-01  QK= 0.5275D-02
 NIT=    7  NFE=    8  F= 0.87961112D+00  WK= 0.1055D-01  QK= 0.5275D-02
 NIT=    8  NFE=   53  F= 0.87961112D+00  WK= 0.1055D-01  QK= 0.5275D-02
 NIT=    9  NFE=   98  F= 0.879611

-5.4004802738569575

In [15]:
include("../src/ctpPOP.jl")
using .ctpPOP

optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        1
Number of nonzeros in inequality constraint Jacobian.:        9
Number of nonzeros in Lagrangian Hessian.............:       23

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        4
        inequality constraints with only lower bounds:        4
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

0.0